# 1. Carga Dataset

In [1]:
import os
import requests

# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')

# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    url = 'https://docs.google.com/uc?export=\download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)


In [2]:
import pandas as pd 

df_inicial=pd.read_csv("data/covertype/covertype_train.csv")
df_inicial

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116198,3150,220,16,285,47,2275,200,253,187,866,Commanche,C7756,1
116199,3125,47,13,234,2,2430,224,212,120,1426,Rawah,C7745,0
116200,3166,152,11,67,0,1275,234,240,136,2404,Rawah,C7202,0
116201,3154,285,14,738,46,6012,181,239,198,1320,Rawah,C7745,1


# 2. Selección de características

Se listan las columnas del dataset

In [3]:
df_inicial.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type',
       'Cover_Type'],
      dtype='object')

Se divide el dataset en 3 dataframes diferentes, los cuales corresponderán a la información númerica, categorica y variable objetivo

In [4]:
df_inicial_numeric=df_inicial[[ 'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways','Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm','Horizontal_Distance_To_Fire_Points']]

df_inicial_categoric=df_inicial[[ 'Wilderness_Area', 'Soil_Type' ]]

target=df_inicial[ 'Cover_Type' ]
target = target.astype(str)

target.value_counts()

1    56720
0    42307
2     7228
6     4045
5     3478
4     1892
3      533
Name: Cover_Type, dtype: int64

con la ayuda de las fucniones SelectKBest y f_classif de la librería se realizará la evaluación de cuales son las principales variables para el modelo

In [5]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif


# Definir cuántas características queremos seleccionar
k = 7  

# Aplicar SelectKBest con f_classif (ANOVA F-score)
selector = SelectKBest(score_func=f_classif, k=k)
X_selected = selector.fit_transform(df_inicial_numeric, target)

# Obtener los puntajes de cada característica
scores = selector.scores_

# Crear un DataFrame con los resultados
feature_scores = pd.DataFrame({
    'Feature': df_inicial_numeric.columns,
    'Score': scores
}).sort_values(by='Score', ascending=False)

# Imprimir los resultados
print("Puntajes de características numéricas:")
print(feature_scores)


Puntajes de características numéricas:
                              Feature         Score
0                           Elevation  31087.079657
5     Horizontal_Distance_To_Roadways   2050.314700
2                               Slope   1559.369722
9  Horizontal_Distance_To_Fire_Points   1452.737911
6                       Hillshade_9am    643.862634
7                      Hillshade_Noon    595.275348
3    Horizontal_Distance_To_Hydrology    488.221292
4      Vertical_Distance_To_Hydrology    241.029561
8                       Hillshade_3pm    190.497164
1                              Aspect     88.987497


Revisando los score resultantes y viendo cuales fueron las variables seleccionadas, se toma la decisión de eliminar las variables Vertical_Distance_To_Hydrology, Hillshade_3pm y Aspect, dado que no aportan al modelo

In [6]:
selected_features = df_inicial_numeric.columns[selector.get_support()]
print("Variables numéricas seleccionadas:", selected_features.tolist())


Variables numéricas seleccionadas: ['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Horizontal_Distance_To_Fire_Points']


In [7]:
df_inicial_numeric=df_inicial_numeric.drop(columns=["Vertical_Distance_To_Hydrology", "Hillshade_3pm" , "Aspect"])

df_variables=pd.concat([df_inicial_numeric, df_inicial_categoric, target ],axis=1)
df_variables


,Elevation,Slope,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,7,67,1015,233,234,1570,Commanche,C7202,1
1,2876,18,485,2495,192,202,1557,Commanche,C7757,1
2,3171,2,277,4374,213,237,1052,Rawah,C7745,0
3,3087,13,190,4774,193,221,752,Rawah,C7745,0
4,2835,10,212,3596,231,242,3280,Rawah,C4744,1
...,...,...,...,...,...,...,...,...,...,...
116198,3150,16,285,2275,200,253,866,Commanche,C7756,1
116199,3125,13,234,2430,224,212,1426,Rawah,C7745,0
116200,3166,11,67,1275,234,240,2404,Rawah,C7202,0
116201,3154,14,738,6012,181,239,1320,Rawah,C7745,1


In [8]:
df_variables['Cover_Type'] = df_variables['Cover_Type'].astype(str).apply(lambda x: f"_{x}")
df_variables

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,7,67,1015,233,234,1570,Commanche,C7202,_1
1,2876,18,485,2495,192,202,1557,Commanche,C7757,_1
2,3171,2,277,4374,213,237,1052,Rawah,C7745,_0
3,3087,13,190,4774,193,221,752,Rawah,C7745,_0
4,2835,10,212,3596,231,242,3280,Rawah,C4744,_1
...,...,...,...,...,...,...,...,...,...,...
116198,3150,16,285,2275,200,253,866,Commanche,C7756,_1
116199,3125,13,234,2430,224,212,1426,Rawah,C7745,_0
116200,3166,11,67,1275,234,240,2404,Rawah,C7202,_0
116201,3154,14,738,6012,181,239,1320,Rawah,C7745,_1


# 3. Data Pipeline

## 3.1 Configurar el contexto interactivo

Se realiza la carga de las librerías necesarias para usar TFX

In [9]:
import tensorflow as tf

# Importar componentes de TFX
from tfx.components import CsvExampleGen, ExampleValidator, SchemaGen, StatisticsGen, Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

2025-03-01 14:30:44.982825: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 14:30:44.993387: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-01 14:30:45.096530: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-01 14:30:45.271367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-01 14:30:45.391522: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

Se configura el contexto interactivo, adicionalmente se indica la ruta en donde se guardarán los artefactos del pipeline

In [10]:
_pipeline_root = './pipeline/'

context = InteractiveContext(pipeline_root=_pipeline_root)

## 3.2. Generando Ejemplos

primero debemos guardar nuestra información en un archivo csv que pueda ser utilizado por TFX

In [11]:
# Definir la ruta donde se guardará el archivo CSV
data_dir = os.path.join(os.getcwd(), "data_tfx")
os.makedirs(data_dir, exist_ok=True)

csv_path = os.path.join(data_dir, "data.csv")

# Guardar el DataFrame como CSV (sin el índice para evitar problemas)
df_variables.to_csv(csv_path, index=False)

Se realiza la transformación de la información al formato requerido por TFX, adicionalmente se ejecuta el componente dentro del pipeline interactivo

In [12]:
example_gen = CsvExampleGen(input_base=data_dir)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Se obtiene el artefacto, en donde se verifica los nombres de los conjuntos en los que se ha dividido la data, adicionalmente la ruta donde se está guardando el pipeline inicial

In [13]:
artifact = example_gen.outputs['examples'].get()[0]

print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: ./pipeline/CsvExampleGen/examples/1


Se verifica la ruta donde se tienen los datos de entrenamiento y el nombre del archivo con la data generada

In [14]:
train_uri = os.path.join(artifact.uri, 'Split-train')

!ls {train_uri}

data_tfrecord-00000-of-00001.gz


Se cargan los ejemplos de entrenamiento en un dataset

In [15]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

Se define una función que permite visualizar los n primeros registros del dataset seleccionado, posteriormente se imprimen los 3 primeros datos

In [16]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

sample_records = get_records(dataset, 3)
pp.pprint(sample_records)    

[{'features': {'feature': {'Cover_Type': {'bytesList': {'value': ['XzE=']}},
                           'Elevation': {'int64List': {'value': ['2991']}},
                           'Hillshade_9am': {'int64List': {'value': ['233']}},
                           'Hillshade_Noon': {'int64List': {'value': ['234']}},
                           'Horizontal_Distance_To_Fire_Points': {'int64List': {'value': ['1570']}},
                           'Horizontal_Distance_To_Hydrology': {'int64List': {'value': ['67']}},
                           'Horizontal_Distance_To_Roadways': {'int64List': {'value': ['1015']}},
                           'Slope': {'int64List': {'value': ['7']}},
                           'Soil_Type': {'bytesList': {'value': ['QzcyMDI=']}},
                           'Wilderness_Area': {'bytesList': {'value': ['Q29tbWFuY2hl']}}}}},
 {'features': {'feature': {'Cover_Type': {'bytesList': {'value': ['XzE=']}},
                           'Elevation': {'int64List': {'value': ['2876']}

2025-03-01 14:31:35.914283: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## 3.3. Estadísticas

A partir del conjunto de ejemplo generado se ejecutaran las estadisticas tanto para el conjunto de entrenamiento como para el de validación

In [17]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [18]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

Ahora veremos las estadísticas generadas para ambos conjuntos

## 3.4. Inferir el esquema

Para poder inferir el esquema, primero debemos isntanciarlo y ejecutarlo

In [19]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Ahora revisando el esquema inferido, vemos que cumple con lo esperado, tenemos 2 variables categoricas y el resto de variables son númericas

In [20]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',STRING,required,,'Cover_Type'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Cover_Type',"'_0', '_1', '_2', '_3', '_4', '_5', '_6'"
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## 3.5. Curando el esquema

In [21]:
import tensorflow_data_validation as tfdv
from tensorflow_metadata.proto.v0 import schema_pb2

# Obtener la ruta del esquema
schema_uri = schema_gen.outputs['schema'].get()[0].uri

# Cargar el esquema en memoria
schema = tfdv.load_schema_text(schema_uri + "/schema.pbtxt")


In [22]:
# Definir un dominio para 'Elevation'
Hillshade_9am_domain = schema_pb2.IntDomain(
    name='Hillshade_9am_domain',min=0,max=255 )
tfdv.set_domain(schema, 'Hillshade_9am', Hillshade_9am_domain)


Hillshade_Noon_domain = schema_pb2.IntDomain(
    name='Hillshade_Noon_domain',min=0,max=255 )
tfdv.set_domain(schema, 'Hillshade_Noon', Hillshade_Noon_domain)

slope_domain = schema_pb2.IntDomain(
    name='slope_domain', min=0, max=90
)
tfdv.set_domain(schema, 'Slope', slope_domain)


In [23]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',STRING,required,,'Cover_Type'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Cover_Type',"'_0', '_1', '_2', '_3', '_4', '_5', '_6'"
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [24]:
tfdv.write_schema_text(schema, schema_uri + "/schema.pbtxt")

## 3.6. Entornos de esquema

Se realiza la carga del esquema creado anteriormente

In [25]:
# Cargar el esquema generado previamente
schema_uri = schema_gen.outputs['schema'].get()[0].uri
schema = tfdv.load_schema_text(schema_uri + "/schema.pbtxt")


Se configuran los entornos de "TRAINING" y "SERVING"

In [26]:
# Configurar los entornos del esquema
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

Se elimina la variable objetivo "Cover_Type" del endorno "Serving"

In [27]:
# Remover Cover_type del entorno SERVING
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')


Se guarda el actualiza el esquema y se visualiza el nuevo esquema

In [28]:
# Guardar el esquema actualizado
tfdv.write_schema_text(schema, schema_uri + "/schema.pbtxt")

# Visualizar el esquema para verificar los cambios
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',STRING,required,,'Cover_Type'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Cover_Type',"'_0', '_1', '_2', '_3', '_4', '_5', '_6'"
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


Finalmente se confirma que ambos entornos fueron creados y que "Cover_Type" no esté incluido en "SERVING"

In [29]:
# Verificar entornos definidos en el esquema
print("Entornos disponibles en el esquema:", schema.default_environment)

# Verificar qué características NO están en SERVING
for feature in schema.feature:
    if "SERVING" in feature.not_in_environment:
        print(f"'{feature.name}' no está en SERVING")



Entornos disponibles en el esquema: ['TRAINING', 'SERVING']
'Cover_Type' no está en SERVING


## 3.7. Genere nuevas estadísticas usando el esquema actualizado

Se realiza la importación del esquema generado anteriormente y se ejecutan las estadisticas nuevamente

In [30]:
from tfx.v1.components import ImportSchemaGen

schema_gen = ImportSchemaGen(schema_file=schema_uri + "/schema.pbtxt")

# Ejecutar el componente para actualizarlo
context.run(schema_gen)

# Crear StatisticsGen pasando el esquema actualizado
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema']  # Pasamos el esquema actualizado
)

# Ejecutar el componente en el contexto de TFX
context.run(statistics_gen)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 5
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Se grafican las estadísticas nuevamente y se observa que la variable Cover_Type sigue apareciendo en formato numérico, a pesar de que el esquema fue modificado para reconocerla como STRING. Esto ocurre porque el cambio realizado en el esquema no altera los datos originales, los cuales siguen almacenados en formato numérico. Para reflejar correctamente la transformación, es necesario convertir Cover_Type a STRING en la fuente de datos antes de su procesamiento en ExampleGen.

In [31]:
context.show(statistics_gen.outputs['statistics'])

## 3.8. Comprobar anomalías

Se realiza la comprobación de anomalías y se detecta un problema relacionado con **Cover_Type**. Esta anomalía ocurre porque, aunque el esquema fue modificado para esperar un **STRING**, los datos originales aún están almacenados en **formato numérico**. Como resultado, `TFDV` identifica una discrepancia entre el tipo de dato esperado y el tipo de dato real presente en el conjunto de datos.

In [32]:
from tfx.components import ExampleValidator

# Crear el componente ExampleValidator con estadísticas y esquema
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],  # Estadísticas generadas
    schema=schema_gen.outputs['schema']  # Esquema actualizado
)

# Ejecutar ExampleValidator en el pipeline de TFX
context.run(example_validator)

# Mostrar las anomalías detectadas
context.show(example_validator.outputs['anomalies'])



Dado que **Cover_Type** es la variable objetivo, no se espera que aparezca en la nueva información que el modelo utilizará en producción. Por ello, se crea un conjunto de datos sin esta variable, utilizando el **10% de la data original**, y se evalúa con el esquema de **"SERVING"**. Como resultado, se confirma que no se detectan anomalías, lo que indica que la estructura de los datos de inferencia es consistente con el esquema de producción.

In [33]:
# Tomar una muestra de los datos de entrenamiento
df_serving = df_variables.sample(frac=0.1, random_state=42).reset_index(drop=True)  # Usamos el 10% de los datos

# Eliminar la columna Cover_Type para simular datos de inferencia
df_serving = df_serving.drop(columns=['Cover_Type'])

serving_stats = tfdv.generate_statistics_from_dataframe(df_serving)


In [34]:
serving_anomalies_with_env = tfdv.validate_statistics(
    serving_stats, schema, environment='SERVING')

tfdv.display_anomalies(serving_anomalies_with_env)


## 3.9. Ingeniería de características

In [43]:
# Set the constants module filename
_covertype_constants_module_file = 'covertype_constants.py'

In [44]:
%%writefile {_covertype_constants_module_file}

# Features with string data types that will be converted to indices
CATEGORICAL_FEATURE_KEYS = ['Soil_Type' , 'Wilderness_Area']

# features númericas que escalados de 0 a 1
NUMERIC_TO_SCALE = ["Hillshade_9am","Hillshade_Noon"]

# features númericas que seran transformadas con z_score
NUMERIC_TO_Z = ["Elevation","Horizontal_Distance_To_Fire_Points","Horizontal_Distance_To_Hydrology","Horizontal_Distance_To_Roadways"]


# Feature that can be grouped into buckets
BUCKET_FEATURE_KEYS = ['Slope']

# Number of buckets used by tf.transform for encoding each bucket feature.
FEATURE_BUCKET_COUNT = {'Slope': 5}

# Feature that the model will predict
Cover_Type_KEY = 'Cover_Type'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Overwriting covertype_constants.py


## 3.10. Función de preprocesamiento

In [45]:
# Set the transform module filename
_covertype_transform_module_file = 'covertype_transform.py'

In [46]:
%%writefile {_covertype_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import covertype_constants

# Unpack the contents of the constants module

_CATEGORICAL_FEATURE_KEYS=covertype_constants.CATEGORICAL_FEATURE_KEYS
_NUMERIC_TO_SCALE=covertype_constants.NUMERIC_TO_SCALE
_NUMERIC_TO_Z=covertype_constants.NUMERIC_TO_Z
_BUCKET_FEATURE_KEYS=covertype_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT=covertype_constants.FEATURE_BUCKET_COUNT
_Cover_Type_KEY=covertype_constants.Cover_Type_KEY
_transformed_name=covertype_constants.transformed_name

# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the range [0,1]
    for key in _NUMERIC_TO_SCALE:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(
            inputs[key])

    # Returns a standardized column with mean 0 and variance 1
    for key in _NUMERIC_TO_Z:
        outputs[_transformed_name(key)] = tft.scale_to_z_score(
            inputs[key])
    
    # Bucketize these features
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(
            inputs[key], _FEATURE_BUCKET_COUNT[key])

    # Convert strings to indices in a vocabulary
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convert the label strings to an index
    outputs[_transformed_name(_Cover_Type_KEY)] = tft.compute_and_apply_vocabulary(inputs[_Cover_Type_KEY])
    
    return outputs


Overwriting covertype_transform.py


## 3.11. Transformar

In [47]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_covertype_transform_module_file))

# Run the component
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying covertype_constants.py -> build/lib
copying covertype_transform.py -> build/lib
installing to /tmp/tmph3u8rcdx
running install
running install_lib
copying build/lib/covertype_constants.py -> /tmp/tmph3u8rcdx
copying build/lib/covertype_transform.py -> /tmp/tmph3u8rcdx
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmph3u8rcdx/tfx_user_code_Transform-0.0+02de1e1a007374b7cb0e39225109fbcee7a2260a01b941d33752

/usr/lib/python3/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+02de1e1a007374b7cb0e39225109fbcee7a2260a01b941d33752eae3c4603933-py3-none-any.whl


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+02de1e1a007374b7cb0e39225109fbcee7a2260a01b941d33752eae3c4603933-py3-none-any.whl


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+02de1e1a007374b7cb0e39225109fbcee7a2260a01b941d33752eae3c4603933-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 8
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [48]:
# Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# List the subdirectories under the uri
os.listdir(transform_graph_uri)

['metadata', 'transformed_metadata', 'transform_fn']

In [49]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [50]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['0']}},
                           'Elevation_xf': {'floatList': {'value': [0.11579835]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.9173228]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.9212598]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [-0.30818257]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [-0.9516831]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [-0.8517185]}},
                           'Slope_xf': {'int64List': {'value': ['0']}},
                           'Soil_Type_xf': {'int64List': {'value': ['1']}},
                           'Wilderness_Area_xf': {'int64List': {'value': ['1']}}}}},
 {'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['0']}},
                           'Elevation

2025-03-01 14:37:41.062508: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
